# Lecture 22: Hidden Markov Models and The Occasionally Dishonest Casino
***

        
<img src="figs/dice_banner.png",width=1000,height=50>        

In [623]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

msize = 200
myfigsize=(12,6)

mycolors = {"blue": "steelblue", "red": "#a76c6e", "green": "#6a9373"}

class DishonestCasino:
    def __init__(self, T=None, E=None, random_state=None, Xhist=None, Ehist=None):
        if random_state: np.random.seed(random_state)
        self.T = T
        if self.T is None:
            self.T = np.array([[0.95, 0.05],[0.05, 0.95]])
        self.E = E
        if self.E is None: 
            self.E = np.array([(1./6)*np.ones(6,), (1./10)*np.ones(6,)]); self.E[1,5] = 0.5
        if Xhist is not None: 
            self.Xhist = Xhist 
        self.Ehist = Ehist
        if Ehist is not None: 
            self.Ehist = Ehist 
            self.num_obs = len(self.Ehist)
        self.num_states = self.T.shape[0]
        
    def roll(self, pmf):
        u = np.random.uniform()
        current_mass = 0
        for ii, mass in enumerate(pmf):
            current_mass += mass
            if u < current_mass:
                return ii
        
    def simulate_data(self, max_t):
        state = 0 
        self.Xhist = np.zeros((max_t,)).astype(int)
        self.Ehist = np.zeros((max_t,)).astype(int)
        for tt in range(max_t):
            self.Xhist[tt] = state
            self.Ehist[tt] = self.roll(self.E[state,:])
            state = self.roll(self.T[state,:])
        self.num_obs = max_t
        
    def report_data(self):
        state_labels=["F", "L"]
        print "".join([state_labels[x] for x in self.Xhist])
        print "".join([str(e+1) for e in self.Ehist])
        
    def report_test(self, z):
        state_labels=["F", "L"]
        print "".join([str(e+1) for e in self.Ehist])
        print "".join([state_labels[x] for x in self.Xhist])
        print "".join([state_labels[val] for val in z])
        
    def forward_backward(self):
        
        # forward pass 
        F = np.ones((self.num_states, self.num_obs+1))
        alpha = np.zeros(self.num_obs+1,)
        alpha[0] = np.sum(F[:,0])
        F[:,0] /= alpha[0]
        for ii in range(self.num_obs-1):
            F[:,ii+1] = np.multiply(self.E[:,self.Ehist[ii]], np.dot(self.T.T, F[:,ii]))
            alpha[ii+1] = np.sum(F[:,ii+1])
            F[:,ii+1] = F[:,ii+1]/alpha[ii+1]
            
        # backward pass 
        B = np.ones((self.num_states, self.num_obs+1))
        for ii in range(self.num_obs-1,-1,-1):
            B[:,ii] = np.dot(self.T, self.E[:,self.Ehist[ii]] * B[:,ii+1])
            
        # inference 
        G = F * B 
        for jj in range(G.shape[1]):
            G[:,jj] /= np.sum(G[:,jj])
            
        return G
    
    def decode(self):
        
        # forward pass 
        V = np.ones((self.num_states, self.num_obs+1))
        V[:,0] /= np.sum(V[:,0])
        ptrs = np.zeros((self.num_states, self.num_obs)).astype(int)
        for ii in range(1,self.num_obs-1):
            TV = self.T * np.row_stack(V[:,ii-1])
            ptrs[:,ii-1] = np.array([np.argmax(TV[:,kk]) for kk in range(self.num_states)])
            V[:,ii] = self.E[:,self.Ehist[ii]] * np.array([np.max(TV[:,kk]) for kk in range(self.num_states)])
            
        # backtrace
        best_path = np.zeros((self.num_obs)).astype(int)
        best_path[-1] = np.argmax(V[:,-1])
        for kk in range(self.num_obs-2,-1,-1):
            best_path[kk] = ptrs[best_path[kk+1],kk+1]
        
        return best_path[:-1]
   

In [632]:
T = np.array([[0.95, 0.05],[0.05, 0.95]])
E = np.array([(1./6)*np.ones(6,), (1./10)*np.ones(6,)]); E[1,5] = 0.5
casino = DishonestCasino()
casino.simulate_data(80)
casino.report_data()
G = casino.forward_backward()

FFFFLLLLLLLLLLLLLFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF
15515635466661666555336613264666565354666564665613563453313123262541143421514612


In [638]:
T = np.array([[0.95, 0.05],[0.05, 0.95]])
E = np.array([(1./6)*np.ones(6,), (1./10)*np.ones(6,)]); E[1,5] = 0.5
casino = DishonestCasino()
casino.simulate_data(80)
path = casino.decode()
casino.report_test(path)

41162621453511466145666225155626665144366334366216661616116231466336366661463462
FFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLFFFLLLLLLLLLLLLLLLLLLLLLLFLLLLLLLLLLLLLLFFFFFF
FFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFF
